# `server/udp/podCommands.js` - from node.js /JavaScript to Python (object)

In [1]:
# find out where we are on the file directory
import os, sys

In [3]:
print( os.getcwd())
print( os.listdir(os.getcwd()))

/home/topolo/PropD/servetheloop/packetDef
['.ipynb_checkpoints', 'reactGS', 'podCommands.ipynb', 'react-GS_node_to_py_to_YAML.ipynb']


The `reactGS` folder "mimics" the actual `react-groundstation` github repository, only copying the file directory structure, but the source code itself (which is a lot) isn't completely copied over.  I wanted to keep these scripts/notebooks/files built on top of that github repository to be separate from the actual working code.  

In [6]:
wherepodCommandsis = os.getcwd()+'/reactGS/server/udp/' 
print(wherepodCommandsis)

/home/topolo/PropD/servetheloop/packetDef/reactGS/server/udp/


## `node.js`/(JavaScript) to `json`; i.e. 

## `node.js`/(JavaScript) $\to$ `json`    

Make a copy of `server/udp/podCommands.js`.  

In this copy, comment out `var chalk = require('chalk')` (this is the only thing you have to do manually).  

Run this in the directory containing your copy of `podCommands.js`:

```  
node traverse_podCommands.js  
```  

This should generate a `json` file `podCmds_lst.json`

# Available `podCommands` as a Python list; `json` to Python list, i.e. `json` $\to$ Python list

In [7]:
import json

In [10]:
f_podCmds_json = open(wherepodCommandsis+'podCmds_lst.json','rb')
rawjson_podCmds = f_podCmds_json.read()
f_podCmds_json.close()

In [16]:
print(type(rawjson_podCmds))
podCmds_lst=json.loads(rawjson_podCmds)
print(type(podCmds_lst))
print(len(podCmds_lst)) # there are 104 available commands for the pod!

<type 'str'>
<type 'list'>
104


In [17]:
for cmd in podCmds_lst: 
    print cmd

GS_Heartbeat
LGU_PositionChange
LGU_SpeedChange
FCUPod_Off
FCUPod_Stop
FCUPod_PowerLatch
setBrakeDevelopmentMode
FCUBrake_DisableDevelopmentMode
FCUBrake_EnableDevelopmentMode
FCUBrake_MoveMotorRAW
FCUBrake_MoveMotorIBeam
FCUBrake_BeginInit
FCUBrake_MLPSetZeroLeftBrake
FCUBrake_MLPSetZeroRightBrake
FCUBrake_MLPSetSpanLeftBrake
FCUBrake_MLPSetSpanRightBrake
FCUStepper_SetMaxAngularAccel
FCUStepper_SetPicoMetersPerRev
FCUStepper_SetMaxRPM
FCUStepper_SetMicroStepResolution
FCUBrake_RequestDevelopmentMode
FCUStreamingControlStart_AccelCalData
FCUStreamingControlStart_AccelFullData
FCUStreamingControlStop_Accel
FCUStreamingControlStart_Brakes
FCUStreamingControlStart_MotorsRaw
FCUStreamingControlStart_Lasers
FCUStreamingControlStart_ForwardLaser
FCUAccel_FineZero
FCUAccel_AutoZero
FCUContrast_StartStream
FCUContrast_StopStream
PowerAStopAllManualDischarging
PowerBStopAllManualDischarging
PowerAPowerLatch
PowerBPowerLatch
PowerAChargeRelayOff
PowerAChargeRelayOn
PowerAStreamingOff
PowerBStre